In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import datetime
import os
import sys
import time

from pprint import pprint

In [3]:
sys.path.append("/home/caleml/main-pe/")

In [21]:
import numpy as np

from data.datasets.h36m import Human36M
from data.loader import BatchLoader
from data.utils import transform, camera
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE

from model import config, measures
from model.networks.multi_branch_model import MultiBranchModel
from model.utils import pose_format, log

## dataset

In [7]:
# local loading
local_h36m_path = '/home/caleml/datasets/h36m'
local_h36m = Human36M(local_h36m_path, dataconf=config.human36m_dataconf, poselayout=pose_format.pa17j3d, topology='frames') 

In [13]:
h36m_val = BatchLoader(local_h36m, 
                       ['frame'],
                       ['pose_w', 'pose_uvd', 'afmat', 'camera'], 
                       VALID_MODE,
                       batch_size=local_h36m.get_length(VALID_MODE), 
                       shuffle=True)

log.printcn(log.OKBLUE, 'Preloading Human3.6M validation samples...')

[x_val], [pw_val, puvd_val, afmat_val, scam_val] = h36m_val[0]

Preloading Human3.6M validation samples...


## model

In [19]:
weights_path = '/home/caleml/pe_experiments/test_from_gpuserver2/weights_036.h5'
eval_model = MultiBranchModel(dim=3, n_joints=17, nb_pose_blocks=1)
eval_model.load_weights(weights_path, pose_only=True)

pose shape (?, 17, 3), vis shape (?, 17, 1), concat shape (?, 17, 4)
pose y_pred shape (?, 17, 4)
Shape (?, ?, ?)
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Stem (Model)                    (None, 32, 32, 576)  1039488     input_19[0][0]                   
__________________________________________________________________________________________________
rBlock1 (Model)                 (None, 32, 32, 576)  1312128     Stem[1][0]                       
__________________________________________________________________________________________________
SepConv1 (Model)                (None, 32, 32, 576)  347904      rBlock1[1][0

In [22]:
scores = eval_human36m_sc_error(eval_model.model, 
                                x_val, 
                                pw_val, 
                                afmat_val,
                                puvd_val[:,0,2], 
                                scam_val,  
                                batch_size=24)

pprint(scores)

8695/8695==============================] - 49s 6ms/sample
Avg. mm. error: 74.2
Final averaged error (mm): 74.217
[74.21665599332593]


## eval func

In [16]:
def eval_human36m_sc_error(model,
                           x,
                           pose_w,
                           afmat,
                           rootz,
                           scam,
                           resol_z=2000.,
                           batch_size=8,
                           map_to_pa17j=None,
                           logdir=None,
                           verbose=True):

    assert len(x) == len(pose_w) == len(afmat) == len(scam)

    num_blocks = len(model.outputs)

    y_true_w = pose_w.copy()
    if map_to_pa17j is not None:
        y_true_w = y_true_w[:, map_to_pa17j, :]
    y_pred_w = np.zeros((num_blocks,) + y_true_w.shape)
    if rootz.ndim == 1:
        rootz = np.expand_dims(rootz, axis=-1)

    pred = model.predict(x, batch_size=batch_size, verbose=1)

    # Move the root joints from GT poses to origin
    y_true_w -= y_true_w[:, 0:1, :]

    if verbose:
        log.printc(log.WARNING, 'Avg. mm. error:')

    lower_err = np.inf
    scores = []

    for b in range(num_blocks):

        if num_blocks > 1:
            y_pred = pred[b]
        else:
            y_pred = pred

        # ??
        y_pred = y_pred[:, :, 0:3]

        # project normalized coordiates to the image plane
        y_pred[:, :, 0:2] = transform.transform_pose_sequence(afmat.copy(), y_pred[:, :, 0:2], inverse=True)

        # Recover the absolute Z
        y_pred[:, :, 2] = (resol_z * (y_pred[:, :, 2] - 0.5)) + rootz
        y_pred_uvd = y_pred[:, :, 0:3]

        # camera inverse projection
        for j in range(len(y_pred_uvd)):
            cam = camera.camera_deserialize(scam[j])
            y_pred_w[b, j, :, :] = cam.inverse_project(y_pred_uvd[j])

        # Move the root joint from predicted poses to the origin
        y_pred_w[b, :, :, :] -= y_pred_w[b, :, 0:1, :]

        err_w = measures.mean_distance_error(y_true_w[:, 0:, :], y_pred_w[b, :, 0:, :])
        scores.append(err_w)
        if verbose:
            log.printc(log.WARNING, ' %.1f' % err_w)

        # Keep the best prediction
        if err_w < lower_err:
            lower_err = err_w

    if verbose:
        log.printcn('', '')

    if logdir is not None:
        np.save('%s/y_pred_w.npy' % logdir, y_pred_w)
        np.save('%s/y_true_w.npy' % logdir, y_true_w)

    log.printcn(log.WARNING, 'Final averaged error (mm): %.3f' % lower_err)

    return scores